### Data Pre-Processing

In [1]:
import re
import csv
import pandas as pd  
import string
import nltk
from gensim.parsing.preprocessing import remove_stopwords

In [2]:
# Loading dataset
pd.options.display.max_colwidth = 50
def load_data():
    data = pd.read_csv("https://raw.githubusercontent.com/a1omar/CRL/main/cybersec_hashtags.csv")
    return data

In [3]:
# Taking a look at data 
twit_df = load_data()
twit_df.head(10)

,id,tweetid,guid,link,author,title,description,pubdate,source,favorite_count,...,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,user_verified,Unnamed: 29
0,1,1450107063217139720,https://twitter.com/ECAatState/statuses/145010...,https://twitter.com/ECAatState/statuses/145010...,ECAatState,RT @MeridianIntl: Today officially marks #Cybe...,Today officially marks #CybersecurityAwareness...,2021-10-18 10:30:40,Twitter for iPhone,0,...,19006707,https://pbs.twimg.com/profile_images/879720269...,29748,1386,84219,2009-01-14 21:11:47,@StateDept\'s Bureau of Educational and Cultur...,"Washington, DC",1.0,NaN
1,2,1450106181117165570,https://twitter.com/JuliePerrine/statuses/1450...,https://twitter.com/JuliePerrine/statuses/1450...,JuliePerrine,Creating strong passwords is a crucial #cybers...,Creating strong passwords is a crucial #cybers...,2021-10-18 10:27:09,CoSchedule,0,...,18243819,https://pbs.twimg.com/profile_images/413869238...,23926,3402,5446,2008-12-19 10:39:53,Founder of Julie Perrine International & All T...,"Indianapolis, Indiana",NaN,NaN
2,3,1450103211952508929,https://twitter.com/craziart/statuses/14501032...,https://twitter.com/craziart/statuses/14501032...,craziart,RT @iExperts10: The importance of Smaller step...,The importance of Smaller steps 👌🏻 #securitya...,2021-10-18 10:15:22,Twitter for iPhone,0,...,238667991,https://pbs.twimg.com/profile_images/143947604...,10872,1392,139,2011-01-15 13:50:32,NaN,Malaysia,NaN,NaN
3,4,1450102926001643523,https://twitter.com/CyberSecurityN8/statuses/1...,https://twitter.com/CyberSecurityN8/statuses/1...,CyberSecurityN8,RT @trueownerhq: TrueOwner gives maximum prote...,TrueOwner gives maximum protection to register...,2021-10-18 10:14:13,forRetweeting,0,...,1142424032794406912,https://pbs.twimg.com/profile_images/119713518...,939517,2,20605,2019-06-22 09:28:09,"The place for InfoSec, CyberSecurity, DevSecOp...",NaN,NaN,NaN
4,5,1450102749870235652,https://twitter.com/sectest9/statuses/14501027...,https://twitter.com/sectest9/statuses/14501027...,sectest9,RT @trueownerhq: TrueOwner gives maximum prote...,TrueOwner gives maximum protection to register...,2021-10-18 10:13:31,auto is the only way it can be,0,...,710123736175783938,https://pbs.twimg.com/profile_images/710735123...,2458115,1,37045,2016-03-16 11:21:07,News Hub! Get the latest Security News & Updates!,"Hyderabad, India",NaN,NaN
5,6,1450102425625505796,https://twitter.com/R___83___O/statuses/145010...,https://twitter.com/R___83___O/statuses/145010...,R___83___O,RT @IYS_GmbH: Want to know what the consequenc...,Want to know what the consequences of a #ranso...,2021-10-18 10:12:14,Twitter for Android,0,...,1178717997919330305,https://pbs.twimg.com/profile_images/132107403...,366623,3831,12142,2019-09-30 13:07:57,‏‏حنونه ومتسامحه,NaN,NaN,NaN
6,7,1450101940923338757,https://twitter.com/trueownerhq/statuses/14501...,https://twitter.com/trueownerhq/statuses/14501...,trueownerhq,TrueOwner gives maximum protection to register...,TrueOwner gives maximum protection to register...,2021-10-18 10:10:18,Twitter for Android,0,...,1423776311684780036,https://pbs.twimg.com/profile_images/142377644...,72,13,7,2021-08-06 18:41:48,Gadget Ownership Gadget #Security Gadget #trac...,NaN,NaN,NaN
7,8,1450101860673667074,https://twitter.com/NebulaITUK/statuses/145010...,https://twitter.com/NebulaITUK/statuses/145010...,NebulaITUK,📱 #MFA helps keep your accounts and data #secu...,📱 #MFA helps keep your accounts and data #secu...,2021-10-18 10:09:59,HubSpot,0,...,1020116743,https://pbs.twimg.com/profile_images/144250255...,464,184,146,2012-12-18 11:13:29,"Based in Bristol & Bath, we pride ourselves on...",01454 534009,NaN,NaN
8,9,1450101746181853191,https://twitter.com/cybersec_feeds/statuses/14...,https://twitter.com/cybersec_feeds/statuses/14...,cybersec_feeds,RT @ProAktivePeople: .@ProAktivePete is speaki...,.@ProAktivePete is speaking @AAG_IT Cyber webi...,2021-10-18 10:09:3

In [4]:
# Finding out the size of the DF and Attribute names
print('Dataset size:',twit_df.shape)
print('Columns are:',twit_df.columns)

Dataset size: (2500, 30)
Columns are: Index(['id', 'tweetid', 'guid', 'link', 'author', 'title', 'description',
       'pubdate', 'source', 'favorite_count', 'retweet_count', 'lang',
       'quoted_text', 'tweet_type', 'in_reply_to_screen_name',
       'in_reply_to_user_id', 'in_reply_to_status_id', 'retweeted_screen_name',
       'retweeted_user_id', 'retweeted_status_id', 'user_id',
       'profile_image_url', 'user_statuses_count', 'user_friends_count',
       'user_followers_count', 'user_created_at', 'user_bio', 'user_location',
       'user_verified', 'Unnamed: 29'],
      dtype='object')


In [5]:
# Finding out more info on DF
twit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       2500 non-null   int64  
 1   tweetid                  2500 non-null   int64  
 2   guid                     2500 non-null   object 
 3   link                     2500 non-null   object 
 4   author                   2500 non-null   object 
 5   title                    2500 non-null   object 
 6   description              2500 non-null   object 
 7   pubdate                  2500 non-null   object 
 8   source                   2500 non-null   object 
 9   favorite_count           2500 non-null   int64  
 10  retweet_count            2500 non-null   int64  
 11  lang                     2500 non-null   object 
 12  quoted_text              15 non-null     object 
 13  tweet_type               2500 non-null   object 
 14  in_reply_to_screen_name 

In [6]:
twit_df

,id,tweetid,guid,link,author,title,description,pubdate,source,favorite_count,...,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,user_verified,Unnamed: 29
0,1,1450107063217139720,https://twitter.com/ECAatState/statuses/145010...,https://twitter.com/ECAatState/statuses/145010...,ECAatState,RT @MeridianIntl: Today officially marks #Cybe...,Today officially marks #CybersecurityAwareness...,2021-10-18 10:30:40,Twitter for iPhone,0,...,19006707,https://pbs.twimg.com/profile_images/879720269...,29748,1386,84219,2009-01-14 21:11:47,@StateDept\'s Bureau of Educational and Cultur...,"Washington, DC",1.0,NaN
1,2,1450106181117165570,https://twitter.com/JuliePerrine/statuses/1450...,https://twitter.com/JuliePerrine/statuses/1450...,JuliePerrine,Creating strong passwords is a crucial #cybers...,Creating strong passwords is a crucial #cybers...,2021-10-18 10:27:09,CoSchedule,0,...,18243819,https://pbs.twimg.com/profile_images/413869238...,23926,3402,5446,2008-12-19 10:39:53,Founder of Julie Perrine International & All T...,"Indianapolis, Indiana",NaN,NaN
2,3,1450103211952508929,https://twitter.com/craziart/statuses/14501032...,https://twitter.com/craziart/statuses/14501032...,craziart,RT @iExperts10: The importance of Smaller step...,The importance of Smaller steps 👌🏻 #securitya...,2021-10-18 10:15:22,Twitter for iPhone,0,...,238667991,https://pbs.twimg.com/profile_images/143947604...,10872,1392,139,2011-01-15 13:50:32,NaN,Malaysia,NaN,NaN
3,4,1450102926001643523,https://twitter.com/CyberSecurityN8/statuses/1...,https://twitter.com/CyberSecurityN8/statuses/1...,CyberSecurityN8,RT @trueownerhq: TrueOwner gives maximum prote...,TrueOwner gives maximum protection to register...,2021-10-18 10:14:13,forRetweeting,0,...,1142424032794406912,https://pbs.twimg.com/profile_images/119713518...,939517,2,20605,2019-06-22 09:28:09,"The place for InfoSec, CyberSecurity, DevSecOp...",NaN,NaN,NaN
4,5,1450102749870235652,https://twitter.com/sectest9/statuses/14501027...,https://twitter.com/sectest9/statuses/14501027...,sectest9,RT @trueownerhq: TrueOwner gives maximum prote...,TrueOwner gives maximum protection to register...,2021-10-18 10:13:31,auto is the only way it can be,0,...,710123736175783938,https://pbs.twimg.com/profile_images/710735123...,2458115,1,37045,2016-03-16 11:21:07,News Hub! Get the latest Security News & Updates!,"Hyderabad, India",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2496,1447416279959105539,https://twitter.com/FortifySolutio1/statuses/1...,https://twitter.com/FortifySolutio1/statuses/1...,FortifySolutio1,"If the password is 7 characters or less, then ...","If the password is 7 characters or less, then ...",2021-10-11 00:18:27,Twitter Web App,0,...,1040221638711828480,https://pbs.twimg.com/profile_images/144541744...,757,514,238,2018-09-13 08:52:19,"Increasing #security awareness, teaching #cybe...","Mumbai, India",NaN,NaN
2496,2497,1447414464228179973,https://twitter.com/India06091186/statuses/144...,https://twitter.com/India06091186/statuses/144...,India06091186,RT @CySecKCoE: Day 10- #Tip10 Stay #cybersecur...,Day 10- #Tip10 Stay #cybersecure and #cyberawa...,2021-10-11 00:11:14,Twitter for Android,0,...,1232756942528864256,https://pbs.twimg.com/profile_images/123275860...,356,876,161,2020-02-26 14:59:09,I\'m Indian,NaN,NaN,NaN
2497,2498,1447403562892349442,https://twitter.com/siddipetcp/statuses/144740...,https://twitter.com/siddipetcp/statuses/144740...,siddipetcp,RT @TelanganaCOPs: Beware of Fake loan apps an...,Beware of Fake loan apps and Be Alert! #CyberA...,2021-10-10 23:27:55,Twitter for iPhone,0,...,820892396523286528,https://pbs.twimg.com/profile_images/125280441...,3111,104,8160,2017-01-16 02:16:13,Official Twitter handle of the Commissioner of...,"Siddipet, Telangana, India",NaN,NaN
2498,2499,1447394817638494211,https://twitter.com/yashjpandit/statuses/14473...,https://twitter.com/ya

In [7]:
# Noticed that one of the columns are repeated guid and link
twit_df.iloc[:,2].equals(twit_df.iloc[:,3])

True

In [8]:
# Dropped 'guid' column
twit_df = twit_df.drop(columns = ["guid","Unnamed: 29"])

In [9]:
twit_df.head(10)

,id,tweetid,link,author,title,description,pubdate,source,favorite_count,retweet_count,...,retweeted_status_id,user_id,profile_image_url,user_statuses_count,user_friends_count,user_followers_count,user_created_at,user_bio,user_location,user_verified
0,1,1450107063217139720,https://twitter.com/ECAatState/statuses/145010...,ECAatState,RT @MeridianIntl: Today officially marks #Cybe...,Today officially marks #CybersecurityAwareness...,2021-10-18 10:30:40,Twitter for iPhone,0,1,...,1450069556303630341,19006707,https://pbs.twimg.com/profile_images/879720269...,29748,1386,84219,2009-01-14 21:11:47,@StateDept\'s Bureau of Educational and Cultur...,"Washington, DC",1.0
1,2,1450106181117165570,https://twitter.com/JuliePerrine/statuses/1450...,JuliePerrine,Creating strong passwords is a crucial #cybers...,Creating strong passwords is a crucial #cybers...,2021-10-18 10:27:09,CoSchedule,0,0,...,0,18243819,https://pbs.twimg.com/profile_images/413869238...,23926,3402,5446,2008-12-19 10:39:53,Founder of Julie Perrine International & All T...,"Indianapolis, Indiana",NaN
2,3,1450103211952508929,https://twitter.com/craziart/statuses/14501032...,craziart,RT @iExperts10: The importance of Smaller step...,The importance of Smaller steps 👌🏻 #securitya...,2021-10-18 10:15:22,Twitter for iPhone,0,5,...,1450100251621302277,238667991,https://pbs.twimg.com/profile_images/143947604...,10872,1392,139,2011-01-15 13:50:32,NaN,Malaysia,NaN
3,4,1450102926001643523,https://twitter.com/CyberSecurityN8/statuses/1...,CyberSecurityN8,RT @trueownerhq: TrueOwner gives maximum prote...,TrueOwner gives maximum protection to register...,2021-10-18 10:14:13,forRetweeting,0,2,...,1450101940923338757,1142424032794406912,https://pbs.twimg.com/profile_images/119713518...,939517,2,20605,2019-06-22 09:28:09,"The place for InfoSec, CyberSecurity, DevSecOp...",NaN,NaN
4,5,1450102749870235652,https://twitter.com/sectest9/statuses/14501027...,sectest9,RT @trueownerhq: TrueOwner gives maximum prote...,TrueOwner gives maximum protection to register...,2021-10-18 10:13:31,auto is the only way it can be,0,2,...,1450101940923338757,710123736175783938,https://pbs.twimg.com/profile_images/710735123...,2458115,1,37045,2016-03-16 11:21:07,News Hub! Get the latest Security News & Updates!,"Hyderabad, India",NaN
5,6,1450102425625505796,https://twitter.com/R___83___O/statuses/145010...,R___83___O,RT @IYS_GmbH: Want to know what the consequenc...,Want to know what the consequences of a #ranso...,2021-10-18 10:12:14,Twitter for Android,0,18,...,1440285074092486656,1178717997919330305,https://pbs.twimg.com/profile_images/132107403...,366623,3831,12142,2019-09-30 13:07:57,‏‏حنونه ومتسامحه,NaN,NaN
6,7,1450101940923338757,https://twitter.com/trueownerhq/statuses/14501...,trueownerhq,TrueOwner gives maximum protection to register...,TrueOwner gives maximum protection to register...,2021-10-18 10:10:18,Twitter for Android,0,2,...,0,1423776311684780036,https://pbs.twimg.com/profile_images/142377644...,72,13,7,2021-08-06 18:41:48,Gadget Ownership Gadget #Security Gadget #trac...,NaN,NaN
7,8,1450101860673667074,https://twitter.com/NebulaITUK/statuses/145010...,NebulaITUK,📱 #MFA helps keep your accounts and data #secu...,📱 #MFA helps keep your accounts and data #secu...,2021-10-18 10:09:59,HubSpot,0,0,...,0,1020116743,https://pbs.twimg.com/profile_images/144250255...,464,184,146,2012-12-18 11:13:29,"Based in Bristol & Bath, we pride ourselves on...",01454 534009,NaN
8,9,1450101746181853191,https://twitter.com/cybersec_feeds/statuses/14...,cybersec_feeds,RT @ProAktivePeople: .@ProAktivePete is speaki...,.@ProAktivePete is speaking @AAG_IT Cyber webi...,2021-10-18 10:09:32,Cyber Security Feed,0,4,...,1450038711304040450,1131854274223366144,https://pbs.twimg.com/profile_images/113185501...,1850580,1,18753,2019-05-24 05:27:42,check out a privacy oriented social media plat...,Internet,NaN
9,10,1450101549162647556,https://twitter.com/aryannegi009/statuses/1450...,aryannegi009,RT @CySecKCoE: Day 16- #Tip16 Stay 

In [10]:
# Checking for duplicate users
# number of rows - number of duplicates
twit_df.shape[0] - twit_df['user_id'].duplicated().sum()

940

In [11]:
# Checking for duplicate tweets
# number of rows - number of duplicates
twit_df.shape[0] - twit_df['description'].duplicated().sum()

981

In [12]:
# Finding who tweets the most
twit_df['author'].value_counts().head(10)

cybersec_feeds     160
CyberSecurityN8    121
sectest9           119
botcybersec        116
BiswajeetRay7       93
fixrnix             79
indiancyberops      54
sshanmugavel        53
CybrainInfosec      48
Sooraj_Sathya       48
Name: author, dtype: int64

In [13]:
# Shows that 36% of the tweets were tweeted by ten people
twit_df['author'].value_counts().cumsum().head(10)

cybersec_feeds     160
CyberSecurityN8    281
sectest9           400
botcybersec        516
BiswajeetRay7      609
fixrnix            688
indiancyberops     742
sshanmugavel       795
CybrainInfosec     843
Sooraj_Sathya      891
Name: author, dtype: int64

### Segmenting the dataset with Users and Tweets

In [14]:
# Display all the text
pd.options.display.max_colwidth = 1000

# Create seperate dataframe for tweets
tweet_df  = pd.DataFrame(twit_df[['author','description']])

In [15]:
tweet_df.head(30)

,author,description
0,ECAatState,"Today officially marks #CybersecurityAwarenessWeek 🎉! As we evolve in the digital world, #cybersecurity plays a big part in our everyday lives. Learn how to enhance your #CyberSmart skills in resources and events here ➡️ https://t.co/8CRQz68NUF https://t.co/hOU637vDXv"
1,JuliePerrine,"Creating strong passwords is a crucial #cybersecurity measure. But that\'s only a piece of the password puzzle. You also need to ensure you\'re managing them safely. This week, we\'re sharing some password management and protection tips &amp; resources. #BeCyberSmart #CyberAware https://t.co/63IBYOBDvB"
2,craziart,The importance of Smaller steps 👌🏻 #securityawareness #securityassessment #cyberattack #informationsecurity #cybersecurityawareness #cybersecurity #infosec #ransomwareattacks #cybercrime #informationsecurity #ransomware #phishing #cloudsecurity #cybersecuritynews https://t.co/XqpZYEGibG
3,CyberSecurityN8,TrueOwner gives maximum protection to registered devices. #findmydevice #SecuritySystems #securitylife #securityservices #securityawareness #phone #Laptop #techgadgets #technews #gadgetstore #gadgetlover #gadget #IMEICheckerPro #onlinesafety #internetsafety #cybersafety https://t.co/hbnkCWurrt
4,sectest9,TrueOwner gives maximum protection to registered devices. #findmydevice #SecuritySystems #securitylife #securityservices #securityawareness #phone #Laptop #techgadgets #technews #gadgetstore #gadgetlover #gadget #IMEICheckerPro #onlinesafety #internetsafety #cybersafety https://t.co/hbnkCWurrt
5,R___83___O,Want to know what the consequences of a #ransomware attack could be for your business? 😱👇 📄Inability to fulfil contractual obligations 💻Complete/partial loss of data 🛠️Operations grind to a halt 📉Irreversible damage to reputation #cyberawareness #infosec #cybersecurity https://t.co/Z7oRASomkX
6,trueownerhq,TrueOwner gives maximum protection to registered devices. #findmydevice #SecuritySystems #securitylife #securityservices #securityawareness #phone #Laptop #techgadgets #technews #gadgetstore #gadgetlover #gadget #IMEICheckerPro #onlinesafety #internetsafety #cybersafety https://t.co/hbnkCWurrt
7,NebulaITUK,"📱 #MFA helps keep your accounts and data #secure, which is why we love Microsoft’s new #passwordless feature. Have you tried it yet? 👇👇 https://t.co/8eqnkx1Sze #cybersecurity #2fa #microsoft #login #cyberaware #security"
8,cybersec_feeds,.@ProAktivePete is speaking @AAG_IT Cyber webinar on 4th Nov - The Cyber Security Landscape: What\'s Going On &amp; How Can You Stay Protected? Sign up here: https://t.co/O8uSSmziAn #CyberSecurity #CyberAware
9,aryannegi009,Day 16- #Tip16 Stay #cybersecure and #cyberaware Do Your Part. #BeCyberSmart #cybersecurity #CybersecurityAwarenessMonth #CyberAwareness @iiscbangalore @ITBTGoK @Startup_Kar @Cyberdost @CybercrimeCID @BlrCityPolice https://t.co/OZfmaQPvsg


### Tweet Preprocessing: 

Twitter cleaning steps

- Remove hyperlinks
- Remove hastags
- Remove @ mentions
- Remove Retweets
- Punctuation

Text cleaning steps

- Make text all lower case
- Remove punctuation
- Remove numerical values
- Remove common non-sensical text (/n &amp)
- Tokenization
- Remove stop words
- Stemming/Lemmatization

In [16]:
# Convert acronyms to words
# Start with Cyber Security related acronyms 

# Create a dictionary for cyber security acronyms 

import requests
from bs4 import BeautifulSoup

# Find a list of acronyms in the cyber security space
r = requests.get('https://www.infosecmatter.com/infosec-glossary/')
soup = BeautifulSoup(r.text, 'html.parser')
acronymTable = soup.find('table', class_ = 'tablepress tablepress-id-3')
acrolist = []
wordslist = []
for acronym in acronymTable.find_all('tbody'):
    rows = acronym.find_all('tr')
    for row in rows:
        acro = row.find('td', class_ = 'column-1').text
        acrolist.append(acro)
        words = row.find('td', class_ = 'column-2').text
        wordslist.append(words)


# Dictionary of acronyms and words in Cyber Security
cyberAcro_dict = dict(zip(acrolist, wordslist)) # regular
cyberAcro_dict_l = {k.lower(): v for k, v in cyberAcro_dict.items()} #lowercase

acrolist_h = ['#' + text for text in acrolist]
cyberAcro_dict_h = dict(zip(acrolist_h, wordslist))

acrolist_l = [text.lower() for text in acrolist_h]     # keys with hashtags and lowercase
cyberAcro_dict_lh = dict(zip(acrolist_l, wordslist))

acrolist_u = [text.upper() for text in acrolist_h]     # keys with hashtags and uppercase
cyberAcro_dict_uh = dict(zip(acrolist_u, wordslist))

In [17]:
# function to convert acronym to word
def key2val(s, dct):
  return ' '.join([dct.get(i, i) for i in s.split()])

In [18]:
# Create function to return string instead of list
# pip install wordninja
import wordninja

def wordninja2Str(x):
    text = ' '.join(map(str, wordninja.split(x)))
    return text

In [19]:
# Create function to clean text for first round 

def clean_Txt_r1(text):
    text = re.sub(r'https?:\/\/\S+', '', text) # Remove hyperlinks
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) # Removes @ mentions
    text = re.sub(r'^RT[\s]+', '', text) # Removing Retweets 
    
    '''Converting acronyms to words''' 
    text = key2val(text, cyberAcro_dict) # mix upper lower acronym without hashtag i.e. IoT internet of things
    text = key2val(text, cyberAcro_dict_h) # mix upper lower acronym with hashtag i.e. IoT internet of things
    text = key2val(text, cyberAcro_dict_uh) # uppercase hashtags i.e. #IOT for Internet of Things
    text = key2val(text, cyberAcro_dict_lh) # lowercase hashtags i.e. #iot for Internet of Things
    
    '''Splitting attached words'''
    old = []
    new = []
    for word in text.split():
        if word.startswith('#'):
            oldword = word
            old.append(oldword)
            newword = word.replace(word,wordninja2Str(word))
            new.append(newword)

    wordsplit = dict(zip(old, new))
    text = key2val(text, wordsplit)
    
    '''Removing hashtags'''
    text = re.sub(r'#', '', text)
    
    return text

round1 = lambda x: clean_Txt_r1(x)

In [20]:
# Round 1 of cleaning the text
tweet_df_cleanR1 = tweet_df.copy()
tweet_df_cleanR1.description = pd.DataFrame(tweet_df_cleanR1.description.apply(round1))

In [21]:
# Show cleaned text
tweet_df_cleanR1.head(30)

,author,description
0,ECAatState,"Today officially marks Cyber security Awareness Week 🎉! As we evolve in the digital world, cyber security plays a big part in our everyday lives. Learn how to enhance your Cyber Smart skills in resources and events here ➡️"
1,JuliePerrine,"Creating strong passwords is a crucial cyber security measure. But that\'s only a piece of the password puzzle. You also need to ensure you\'re managing them safely. This week, we\'re sharing some password management and protection tips &amp; resources. Be Cyber Smart Cyber Aware"
2,craziart,The importance of Smaller steps 👌🏻 security awareness security assessment cyber attack information security cyber security awareness cyber security info sec ransom ware attacks cyber crime information security ransom ware phishing cloud security cyber security news
3,CyberSecurityN8,TrueOwner gives maximum protection to registered devices. find my device Security Systems security life security services security awareness phone Laptop tech gadgets tech news gadget store gadget lover gadget I MEI Checker Pro online safety internet safety cyber safety
4,sectest9,TrueOwner gives maximum protection to registered devices. find my device Security Systems security life security services security awareness phone Laptop tech gadgets tech news gadget store gadget lover gadget I MEI Checker Pro online safety internet safety cyber safety
5,R___83___O,Want to know what the consequences of a ransom ware attack could be for your business? 😱👇 📄Inability to fulfil contractual obligations 💻Complete/partial loss of data 🛠️Operations grind to a halt 📉Irreversible damage to reputation cyber awareness info sec cyber security
6,trueownerhq,TrueOwner gives maximum protection to registered devices. find my device Security Systems security life security services security awareness phone Laptop tech gadgets tech news gadget store gadget lover gadget I MEI Checker Pro online safety internet safety cyber safety
7,NebulaITUK,📱 Multi-factor Authentication helps keep your accounts and data secure which is why we love Microsoft’s new password less feature. Have you tried it yet? 👇👇 cyber security 2-factor Authentication microsoft login cyber aware security
8,cybersec_feeds,. is speaking Cyber webinar on 4th Nov - The Cyber Security Landscape: What\'s Going On &amp; How Can You Stay Protected? Sign up here: Cyber Security Cyber Aware
9,aryannegi009,Day 16- Tip 16 Stay cyber secure and cyber aware Do Your Part. Be Cyber Smart cyber security Cyber security Awareness Month Cyber Awareness


In [22]:
# Create function to take out emojis from text

def remove_emojis(text):
    emoj_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                           "]+", flags=re.UNICODE)
    
    return emoj_pattern.sub(r"",text)


In [23]:
# Create function to clean text for second round 

def clean_Txt_r2(text):
    text = " ".join(remove_emojis(text).split()) # removing emojis
    text = text.replace('&amp;','and') # replace '&' with and
    text = re.sub(r'&[A-Za-z0-9_]+','',text) # removing &gt;&gt
    text = re.sub('\n', '', text) # remove non-sensical text
    text = re.sub('\w*\d\w*', '', text) # remove words and numbers
    
    return text

round2 = lambda x: clean_Txt_r2(x)

In [24]:
# Round 2 of cleaning the text
tweet_df_cleanR2 = tweet_df_cleanR1.copy()
tweet_df_cleanR2.description = pd.DataFrame(tweet_df_cleanR2.description.apply(round2))

In [25]:
pd.set_option('display.max_rows', tweet_df_cleanR2.shape[0]+1)
tweet_df_cleanR2.head(30)

,author,description
0,ECAatState,"Today officially marks Cyber security Awareness Week ! As we evolve in the digital world, cyber security plays a big part in our everyday lives. Learn how to enhance your Cyber Smart skills in resources and events here"
1,JuliePerrine,"Creating strong passwords is a crucial cyber security measure. But that\'s only a piece of the password puzzle. You also need to ensure you\'re managing them safely. This week, we\'re sharing some password management and protection tips and resources. Be Cyber Smart Cyber Aware"
2,craziart,The importance of Smaller steps security awareness security assessment cyber attack information security cyber security awareness cyber security info sec ransom ware attacks cyber crime information security ransom ware phishing cloud security cyber security news
3,CyberSecurityN8,TrueOwner gives maximum protection to registered devices. find my device Security Systems security life security services security awareness phone Laptop tech gadgets tech news gadget store gadget lover gadget I MEI Checker Pro online safety internet safety cyber safety
4,sectest9,TrueOwner gives maximum protection to registered devices. find my device Security Systems security life security services security awareness phone Laptop tech gadgets tech news gadget store gadget lover gadget I MEI Checker Pro online safety internet safety cyber safety
5,R___83___O,Want to know what the consequences of a ransom ware attack could be for your business? Inability to fulfil contractual obligations Complete/partial loss of data Operations grind to a halt Irreversible damage to reputation cyber awareness info sec cyber security
6,trueownerhq,TrueOwner gives maximum protection to registered devices. find my device Security Systems security life security services security awareness phone Laptop tech gadgets tech news gadget store gadget lover gadget I MEI Checker Pro online safety internet safety cyber safety
7,NebulaITUK,Multi-factor Authentication helps keep your accounts and data secure which is why we love Microsoft’s new password less feature. Have you tried it yet? cyber security -factor Authentication microsoft login cyber aware security
8,cybersec_feeds,. is speaking Cyber webinar on Nov - The Cyber Security Landscape: What\'s Going On and How Can You Stay Protected? Sign up here: Cyber Security Cyber Aware
9,aryannegi009,Day - Tip Stay cyber secure and cyber aware Do Your Part. Be Cyber Smart cyber security Cyber security Awareness Month Cyber Awareness


In [26]:
# Create function to clean text for third round

def clean_Txt_r3(text):
    text = text.replace("\\","") # removing backslash
    text = remove_stopwords(text) # take out stopwords
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # take out punctuation
    text = re.sub('[‘’“”…]', '', text) # get rid of extra '' and ""
    text = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",text) if s]) # split attached words that have capitalization
    text = text.lower() # make tweets lower case
    text = ' '.join( [w for w in text.split() if len(w)>1] ) # removing single letters
    text = " ".join(text.split()) # removing unnecessary white spaces 
    return text

round3 = lambda x: clean_Txt_r3(x)

In [27]:
tweet_df_cleanR3 = tweet_df_cleanR2.copy()
tweet_df_cleanR3.description = tweet_df_cleanR2.description.apply(round3)

In [28]:
tweet_df_cleanR3.head(30)

,author,description
0,ECAatState,today officially marks cyber security awareness week as evolve digital world cyber security plays big everyday lives learn enhance cyber smart skills resources events
1,JuliePerrine,creating strong passwords crucial cyber security measure but thats piece password puzzle you need ensure youre managing safely this week were sharing password management protection tips resources be cyber smart cyber aware
2,craziart,the importance smaller steps security awareness security assessment cyber attack information security cyber security awareness cyber security info sec ransom ware attacks cyber crime information security ransom ware phishing cloud security cyber security news
3,CyberSecurityN8,true owner gives maximum protection registered devices device security systems security life security services security awareness phone laptop tech gadgets tech news gadget store gadget lover gadget mei checker pro online safety internet safety cyber safety
4,sectest9,true owner gives maximum protection registered devices device security systems security life security services security awareness phone laptop tech gadgets tech news gadget store gadget lover gadget mei checker pro online safety internet safety cyber safety
5,R___83___O,want know consequences ransom ware attack business inability fulfil contractual obligations completepartial loss data operations grind halt irreversible damage reputation cyber awareness info sec cyber security
6,trueownerhq,true owner gives maximum protection registered devices device security systems security life security services security awareness phone laptop tech gadgets tech news gadget store gadget lover gadget mei checker pro online safety internet safety cyber safety
7,NebulaITUK,multifactor authentication helps accounts data secure love microsofts new password feature have tried yet cyber security factor authentication microsoft login cyber aware security
8,cybersec_feeds,speaking cyber webinar nov the cyber security landscape whats going on how can you stay protected sign here cyber security cyber aware
9,aryannegi009,day tip stay cyber secure cyber aware do your part be cyber smart cyber security cyber security awareness month cyber awareness


In [29]:
# Checking before deleting duplicates
len(tweet_df_cleanR3)

2500

In [30]:
tweet_df_cleanR3.duplicated(subset = "description").sum()

1643

In [31]:
tweet_df_cleanR3.drop_duplicates(subset='description',inplace=True)

In [32]:
len(tweet_df_cleanR3)

857

## Tokenization and Lemmatization

In [33]:
#important libraries for preprocessing using NLTK
import nltk
#nltk.download
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

In [34]:
def lemmatize_token(tweet):
    stop_words = set(stopwords.words('english'))
    tokenized = word_tokenize(tweet)
    filtered = [w for w in tokenized if not w in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized = []
    for word in filtered:
        lemmatized.append(lemmatizer.lemmatize(word))
    lemmatized = ' '.join(lemmatized)
    return lemmatized

In [35]:
tweet_df_cleanR4 = tweet_df_cleanR3.copy()
tweet_df_cleanR4.description = tweet_df_cleanR3.description.map(lambda x: lemmatize_token(x))

In [36]:
tweet_df_cleanR4.head(30)

,author,description
0,ECAatState,today officially mark cyber security awareness week evolve digital world cyber security play big everyday life learn enhance cyber smart skill resource event
1,JuliePerrine,creating strong password crucial cyber security measure thats piece password puzzle need ensure youre managing safely week sharing password management protection tip resource cyber smart cyber aware
2,craziart,importance smaller step security awareness security assessment cyber attack information security cyber security awareness cyber security info sec ransom ware attack cyber crime information security ransom ware phishing cloud security cyber security news
3,CyberSecurityN8,true owner give maximum protection registered device device security system security life security service security awareness phone laptop tech gadget tech news gadget store gadget lover gadget mei checker pro online safety internet safety cyber safety
5,R___83___O,want know consequence ransom ware attack business inability fulfil contractual obligation completepartial loss data operation grind halt irreversible damage reputation cyber awareness info sec cyber security
7,NebulaITUK,multifactor authentication help account data secure love microsofts new password feature tried yet cyber security factor authentication microsoft login cyber aware security
8,cybersec_feeds,speaking cyber webinar nov cyber security landscape whats going stay protected sign cyber security cyber aware
9,aryannegi009,day tip stay cyber secure cyber aware part cyber smart cyber security cyber security awareness month cyber awareness
16,911CyberMarket,national cyber security alliance ncsa educating empowering global digital society free online security checkup tool many cyber security vendor offer free security tool security awareness technology small buisness management
17,GNYHA,hospital health system use health insurance portability accountability act security rule toolkit help implement ass security rule requirement cyber aware


In [37]:
# Amount of duplicates
tweet_df_cleanR4.duplicated(subset = 'description').sum()

3

In [38]:
# Show duplicates
tweet_df_cleanR4[tweet_df_cleanR4.duplicated(subset = 'description',keep = False)]

,author,description
781,CyberSecurityN8,protected ransomware attack step protect phishing cyber awareness ransom ware cyber security cyber smart cyber security awareness month
1413,Micheal74606768,rt cyber aware info sec data security cyber attack cyber security awareness month cyber security world st software service governance risk compliance fresh grc governance risk compliance reg tech world ready cyber war cyber bullying
1423,Micheal74606768,rt cyber aware info sec data security cyber attack cyber security awareness month cyber security world st software service governance risk compliance fresh grc governance risk compliance reg tech world ready cyber war cyber bullying
2080,sondravet22,cyber smart
2100,NXTsoft,protected ransomware attack step protect phishing cyber awareness ransom ware cyber security cyber smart cyber security awareness month
2415,cameronbrister,cyber smart


In [39]:
# Remove duplicates
tweet_df_cleanR4.drop_duplicates(subset='description',inplace=True)

In [40]:
tweet_df_cleanR4.drop_duplicates(inplace=True)
len(tweet_df_cleanR4)

854

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cv = CountVectorizer(stop_words='english')
word_count=cv.fit_transform(tweet_df_cleanR4.description.tolist())
word_count.shape

(854, 2879)

In [42]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

In [43]:
# Least important words
df_idf.sort_values(by=['idf_weights']).head(40)

,idf_weights
cyber,1.071502
security,1.239356
awareness,1.275669
aware,1.901777
month,1.936971
smart,2.308684
attack,2.603607
data,2.603607
sec,2.663505
information,2.795274


In [44]:
# Most important words 
df_idf.sort_values(by=['idf_weights']).tail(40)

,idf_weights
helpful,7.057954
hello,7.057954
heightened,7.057954
heavily,7.057954
havent,7.057954
hovering,7.057954
incredible,7.057954
hua,7.057954
hub,7.057954
including,7.057954
